In [4]:
import requests, re

### Data Gathering

In [5]:
def get_revisions(Property,rvcontinue=None):
    """
    Input:
    Property,rvcontinue
    
    Output:
    revisions, rvcontinue
    """
    URL = "https://www.wikidata.org/w/api.php"
    # defining a params dict for the parameters to be sent to the API
    PARAMS = {"action":"query",
              "prop":"revisions",
              "format":"json",
              "rvlimit":"50",
              "titles":"Property:{}".format(Property),
              "rvdir":"newer",
              "rvcontinue": rvcontinue}
    
    # sending get request and saving the response as response object
    r = requests.get(url = URL,params = PARAMS)

    # extracting data in json format
    data = r.json()
    
    # Get revisions
    pages = data['query']['pages']
    keys = pages.keys()
    revisions = [pages[k]['revisions'] for k in keys]
    rvcontinue = data['continue']['rvcontinue'] if 'continue' in data else None
    
    return revisions, rvcontinue

In [6]:
def get_all_revisions(Property,rvcontinue=None):
    
    all_revisions = []
        
    while True:
        revisions, rvcontinue = get_revisions(Property,rvcontinue)
        all_revisions += revisions[0]
        
        if rvcontinue is None:
            break

    return all_revisions

In [7]:
all_revisions = get_all_revisions('P3960')

### Data parsing


|Attribute |Add| Update |Remove|
|---|---|---|---|
|label |wbsetlabel-add| wbsetlabel-set| wbsetlabel-remove|
|description |wbsetdescription-add| wbsetdescription-set| wbsetdescription-remove|
|alias |wbsetaliases-add |wbsetaliases-add-remove| wbsetaliases-remove|
Edit message for changes related to labels, descriptions and aliases of Wikidata
properties

In [28]:
string = all_revisions[20]['comment']
tag_pattern = "(?<=\/\*)(.*)(?=\*\/)"
description_pattern = "(?<=\*\/)[^\]]+"

In [29]:
attributes_dict = {"label":"wbsetlabel",
                 "description":"wbsetdescription",
                 "alias":"wbsetaliases"}

actions_dict = {"Add":"-add",
               "Update":"-update",
               "Remove":"-remove"}

In [30]:
tag = re.findall(tag_pattern, string)[0].strip()
description =re.findall(description_pattern, string)[0].strip()

In [39]:
print(tag)

wbsetlabel-add:1|hu


In [37]:
[k for k,v in attributes_dict.items() if v in tag]

['label']

In [38]:
[k for k,v in actions_dict.items() if v in tag]

['Add']